In [1]:
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from datetime import datetime

In [21]:
# import the web scraping content
chinese_title = open('chinese_titles.txt','r')
soup = BeautifulSoup(chinese_title, "html.parser")

# create the main dataframe
title=[]; author=[];  publisher=[]; appearance=[]

for i in soup.find_all('p'):
  title.append(i.text)
  author.append(i.next_sibling.text)
  publisher.append(i.next_sibling.next_sibling.text)
  appearance.append(i.parent.parent.next_sibling.text)

df = pd.DataFrame(list(zip(title, author, publisher, appearance)))

df.columns=['title','author','publisher','appearance']
df['api'] = 'https://www.googleapis.com/books/v1/volumes?q=' + df['title'] +'+inauthor:'+df['author']

In [56]:
# call google books api to retrive additional data
lst_cat=[]; lst_des=[]; lst_pt=[]; lst_date=[]; lst_pub=[]; lst_isbn=[]

for i in range(len(df)):
  json_rs = requests.get(df['api'][i])
  json_pd = json.loads(json_rs.text)
  
  try:
    lst_cat.append(json_pd['items'][0]['volumeInfo']['categories'])
  except KeyError:
    lst_cat.append(np.nan)
  
  try:
    lst_des.append(json_pd['items'][0]['volumeInfo']['description'])
  except KeyError:
    lst_des.append(np.nan)

  try:
    lst_pt.append(json_pd['items'][0]['volumeInfo']['printType'])
  except KeyError:
    lst_pt.append(np.nan)

  try:
    lst_date.append(json_pd['items'][0]['volumeInfo']['publishedDate'])
  except KeyError:
    lst_date.append(np.nan)

  try:
    lst_pub.append(json_pd['items'][0]['volumeInfo']['publisher'])
  except KeyError:
    lst_pub.append(np.nan)

  try:
    lst_isbn.append(json_pd['items'][0]['volumeInfo']['industryIdentifiers'][0]['identifier'])
  except KeyError:
    lst_isbn.append(np.nan)

# create the api dataframe
df_api = pd.DataFrame(list(zip(lst_cat, lst_des, lst_pt, lst_date, lst_pub, lst_isbn)))
df_api.columns=['category','description','pub_type','pub_date','publisher_v','isbn']

In [58]:
# merge dataframes
df_merge = df.join(df_api, how='outer')
df_merge.to_csv('df_merge.csv')